## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Data Load

In [2]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_28_1.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_28_1.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']

In [4]:
print(f"IVF_X shape: {IVF_X.shape}")
print(f"IVF_test shape: {IVF_test.drop('ID', axis=1).shape}")

IVF_X shape: (250052, 73)
IVF_test shape: (87891, 73)


### 인코딩 

In [5]:
IVF_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [7]:
# 모든 범주형 변수를 문자열로 변환
IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])

## Modeling

In [8]:
# 데이터 분할
IVF_X_train, IVF_X_test, IVF_y_train, IVF_y_test = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)

### IVF 데이터

In [9]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 9000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),

        'random_state': 42,
        'scale_pos_weight': 2.827,
        'objective': 'binary',
        'metric': 'auc',
        'verbose': -1   
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(IVF_X_train, IVF_y_train)
    
    y_pred_proba = model.predict_proba(IVF_X_test)[:, 1]
    
    auc = roc_auc_score(IVF_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=700)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-13 23:36:36,345] A new study created in memory with name: no-name-da11a4d4-b3be-435a-b5a1-6825a72ba32e
[I 2025-02-13 23:51:14,528] Trial 0 finished with value: 0.7032321710212296 and parameters: {'n_estimators': 8958, 'learning_rate': 0.0021752346318033096, 'num_leaves': 1646, 'min_child_samples': 56, 'subsample': 0.7341453844769206, 'reg_alpha': 0.018547744701367094, 'reg_lambda': 0.24767499756892744}. Best is trial 0 with value: 0.7032321710212296.
[I 2025-02-13 23:57:52,289] Trial 1 finished with value: 0.7280385498936156 and parameters: {'n_estimators': 4387, 'learning_rate': 0.0001178629174262027, 'num_leaves': 1730, 'min_child_samples': 23, 'subsample': 0.34085020801002286, 'reg_alpha': 0.0018180037779387558, 'reg_lambda': 0.00179723223235004}. Best is trial 1 with value: 0.7280385498936156.
[I 2025-02-14 00:02:41,849] Trial 2 finished with value: 0.6772830236544253 and parameters: {'n_estimators': 7936, 'learning_rate': 0.08620797169020396, 'num_leaves': 2029, 'min_ch

Best trial:
  Value: 0.7398259139347938
  Params: 
    n_estimators: 2497
    learning_rate: 0.005927840513532174
    num_leaves: 35
    min_child_samples: 43
    subsample: 0.9865202595019962
    reg_alpha: 8.608360045823542
    reg_lambda: 0.6059745556386879


.